In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  

In [2]:
#Download Postgress driver to connect
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-30 20:42:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.52MB/s    in 0.2s    

2022-08-30 20:42:03 (5.52 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big_Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [27]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [28]:
#Count the number of records (rows) in the dataset.
df.count()

960872

Create Dataframe to tables

Review_id_table

In [29]:
#Create review_id_table w/ columns review_id, customer_id, product_id, product_parent, review_date
review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370|2015-08-31 00:00:00|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722|2015-08-31 00:00:00|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390|2015-08-31 00:00:00|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625|2015-08-31 00:00:00|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221|2015-08-31 00:00:00|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695|2015-08-31 00:00:00|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279|2015-08-31 00:00:00|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113|2015-08-31 00:00:00|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892|2015-08-31 00:00:00|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407|2015-08-31 00:00:00|
+--------------+---------

Products_table

In [31]:
#Create products_table w/product_id, product_title unique values
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
|B00NIDA43Y|GuTe Classic Skel...|
|B008EQDDPQ|Nautica Men's N13...|
|B004VRBZ66|Timex Men's T2N63...|
|B009BEO81I|        Fossil Riley|
|B008B39MTI|XOXO Women's XO55...|
+----------+--------------------+
only showing top 10 rows



Customers_table

In [26]:
#Create customers table w customer_id and customer_count
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df.show(10)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|    1567510|                 1|
|   19502021|                 1|
|   12819130|                 1|
|   35329257|                 2|
|     108460|                 1|
|    5453476|                 1|
|   29913055|                 1|
|   30717305|                 1|
|    1570030|                 1|
|   19032020|                 1|
+-----------+------------------+
only showing top 10 rows



Vine_table

In [9]:
#Create vine_table w/ review_id, star_rating, helpful_votes, total_votes, vine
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [16]:
#Write dataframe to database
mode = "append"
jdbc_url="jdbc:postgresql://{endpoint}/postgres"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [18]:
#Review_id to table
review_id.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [19]:
#Product_df to table
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
#Customer_df to table
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [21]:
#Vine_df to table
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)